Importing libs

In [20]:
import pandas as pd
import re
import os 
import json
import string
import pyparsing
import warnings
import mdtex2html
import numpy as np
from langdetect import detect, DetectorFactory
from bs4 import BeautifulSoup
from cleantext import clean

In [10]:
DATA_PATH = "../data/"
LIST = ['regression','classification','clustering','computer vision','nlp','reinforcement learning']
PATTERN = r'[' + string.punctuation +'\n' ']'
COMMENT_FILTER = pyparsing.pythonStyleComment.suppress()
RMV_URL = re.compile(r'https?://\S+|www\.\S+')
RMV_EQU = r"(\$+)(?:(?!\1)[\s\S])*\1"
DetectorFactory.seed = 0
warnings.filterwarnings('ignore')

Construct a new dataframe where cleaned cells are stored

In [11]:
df = pd.DataFrame(columns=['cell-type','source','title','tag'])

Function that clean cells according to the type

In [12]:
def cleaning_ntbk(df,raw_df,item,category):
    for i in raw_df.index:

        #CLEANING FOR MARKDOWN CELLS 

        if raw_df.loc[i]['cell_type'] == 'markdown':
             
            ''' 
            # Equations remover
            raw_df.loc[i]['source'] = re.sub(RMV_EQU," ",str(raw_df.loc[i]['source']), 0, re.MULTILINE)
            # Latex converter to HTML
            raw_df.loc[i]['source'] = mdtex2html.convert(mdtex2html.convert(str(raw_df.loc[i]['source'])))
            # HTML's tags remover
            raw_df.loc[i]['source'] = BeautifulSoup(str(raw_df.loc[i]['source']), "lxml").text
            # URLs - Names of authors - Punctuation & special caracters Remover
            raw_df.loc[i]['source'] = re.sub(PATTERN,'',re.sub(r'(\s)@\w+', r'\1',RMV_URL.sub(r'',str(raw_df.loc[i]['source']))))
            # Emojis Remover
            raw_df.loc[i]['source'] = clean(re.sub(PATTERN,'',re.sub(r'(\s)@\w+', r'\1',RMV_URL.sub(r'',BeautifulSoup(mdtex2html.convert(mdtex2html.convert(re.sub(RMV_EQU," ",str(raw_df.loc[i]['source']), 0, re.MULTILINE))), "lxml").text))), no_emoji=True)
            '''
            
            df.loc[len(df)]=[raw_df.loc[i]['cell_type'],clean(re.sub(PATTERN,'',re.sub(r'(\s)@\w+', r'\1',RMV_URL.sub(r'',BeautifulSoup(mdtex2html.convert(mdtex2html.convert(re.sub(RMV_EQU," ",str(raw_df.loc[i]['source']), 0, re.MULTILINE))), "lxml").text))), no_emoji=True) ,item,category]
        

        #CLEANING FOR CODE CELLS
        else :
            df.loc[len(df)]=[raw_df.loc[i]['cell_type'],COMMENT_FILTER .transformString(raw_df.loc[i]['source']),item,category]


Function to test if its an english notebook

In [14]:
def browse_by_folder(data,category):
    test = os.listdir(DATA_PATH+category)
    for item in test:
        if item.endswith(".py") or item.endswith(".irnb") or item.endswith(".R") or item.endswith(".Rmd"):
            os.remove(os.path.join(DATA_PATH+category, item))
        else :
            with open(DATA_PATH+category+"/"+item, encoding="utf8") as jsonfile:
                data_ = json.load(jsonfile)
            raw_df = pd.json_normalize(data_['cells'])
            cleaning_ntbk(df,raw_df,item,category)

In [15]:
for category in LIST:
    browse_by_folder(df,category)

In [16]:
df

,cell-type,source,title,tag
0,code,import pandas as pd\nimport numpy as np\nimpor...,0-9-try-better-parameters-better-score.ipynb,regression
1,markdown,try to overfit more pls upvote if you forklike...,0-9-try-better-parameters-better-score.ipynb,regression
2,markdown,sub1 0869,0-9-try-better-parameters-better-score.ipynb,regression
3,markdown,credit very simple code with score 0886 by aya...,0-9-try-better-parameters-better-score.ipynb,regression
4,code,import pandas as pd\nimport numpy as np\nfrom ...,0-9-try-better-parameters-better-score.ipynb,regression
...,...,...,...,...
353221,code,"\nPOI_data = gpd.read_file(""../input/geospatia...",your-first-map.ipynb,reinforcement learning
353222,markdown,next we create a map from all four geodatafram...,your-first-map.ipynb,reinforcement learning
353223,code,"\nax = counties.plot(figsize=(10,10), color='n...",your-first-map.ipynb,reinforcement learning
353224,markdown,it looks like the northeastern part of the sta...,your-first-map.ipynb,reinforcement learning


In [17]:
df.to_csv(DATA_PATH+'cleaned_ntb.csv')

In [24]:
def test_lang_en(data_ntb):
    full_text_md = ''

    for i in data_ntb.index:
        if data_ntb.loc[i]['cell-type'] == "markdown":
            full_text_md += str(data_ntb.loc[i]['source'])
    
    
    if detect(full_text_md) == 'en' :
        return True

In [27]:
def delete_non_en_ntb(data):
    groups = data.groupby('title')
    List_ntb = np.unique(data['title'].tolist()).tolist()
    for ntb in List_ntb :
        try:
            if test_lang_en(groups.get_group(ntb)) != True :
                data.drop(data.index[data['title'] == ntb], inplace=True)
        except Exception as e:
                print(e)

In [28]:
delete_non_en_ntb(df)

No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features i

In [29]:
df.to_csv(DATA_PATH+'cleaned_ntb_only_en.csv')